# ConnorsReversal Strategy Batch Optimization

这个notebook用于对多个交易对进行ConnorsReversal策略的批量优化。

In [1]:
# 导入必要的库
import os
import pandas as pd
from datetime import datetime
from glob import glob
import numpy as np
from backtesting import Backtest
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output  # 新增：导入 clear_output

In [2]:
def load_and_resample_data(symbol, start_date, end_date, source_timeframe='1m', target_timeframe='30min', data_path=r'\\znas\Main\futures'):
    """
    加载并重采样期货数据
    """
    # 生成日期范围
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # 准备存储所有数据的列表
    all_data = []
    
    # 标准化交易对名称
    formatted_symbol = symbol.replace('/', '_').replace(':', '_')
    if not formatted_symbol.endswith('USDT'):
        formatted_symbol = f"{formatted_symbol}USDT"
    
    # 遍历每一天
    for date in date_range:
        date_str = date.strftime('%Y-%m-%d')
        # 构建文件路径
        file_path = os.path.join(data_path, date_str, f"{date_str}_{formatted_symbol}_USDT_{source_timeframe}.csv")
        
        try:
            if os.path.exists(file_path):
                # 读取数据
                df = pd.read_csv(file_path)
                df['datetime'] = pd.to_datetime(df['datetime'])
                all_data.append(df)
            else:
                print(f"文件不存在: {file_path}")
        except Exception as e:
            print(f"读取文件出错 {file_path}: {str(e)}")
            continue
    
    if not all_data:
        raise ValueError(f"未找到 {symbol} 在指定日期范围内的数据")
    
    # 合并所有数据
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df = combined_df.sort_values('datetime')
    
    # 设置时间索引
    combined_df.set_index('datetime', inplace=True)
    
    # 重采样到目标时间周期
    resampled = combined_df.resample(target_timeframe).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }).dropna()  # 立即删除NaN值
    
    # 转换为backtesting.py格式
    backtesting_df = pd.DataFrame({
        'Open': resampled['open'],
        'High': resampled['high'],
        'Low': resampled['low'],
        'Close': resampled['close'],
        'Volume': resampled['volume']
    })
    
    # 确保所有数据都是数值类型并删除任何无效值
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        backtesting_df[col] = pd.to_numeric(backtesting_df[col], errors='coerce')
    
    # 最终清理
    backtesting_df = backtesting_df.dropna()
    
    return backtesting_df

In [3]:
def get_all_symbols(data_path, date_str):
    """获取指定日期目录下的所有交易对"""
    daily_path = os.path.join(data_path, date_str)
    if not os.path.exists(daily_path):
        return []
    
    files = glob(os.path.join(daily_path, f"{date_str}_*_USDT_1m.csv"))
    symbols = []
    for file in files:
        # 从文件名提取交易对名称
        filename = os.path.basename(file)
        symbol = filename.split('_')[1]
        if symbol not in symbols:
            symbols.append(symbol)
    return symbols

def verify_data_completeness(symbol, start_date, end_date, data_path):
    """验证交易对在指定时间段内的数据完整性"""
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    for date in date_range:
        date_str = date.strftime('%Y-%m-%d')
        file_path = os.path.join(data_path, date_str, f"{date_str}_{symbol}_USDT_1m.csv")
        if not os.path.exists(file_path):
            return False
    return True

In [4]:
# 使用自定义评分函数(推荐)
def custom_score(stats):
    """
    自定义评分函数，综合考虑多个指标
    """
    # 获取关键指标
    sharpe = stats['Sharpe Ratio']
    max_dd = stats['Max. Drawdown [%]']
    ret = stats['Return [%]']
    win_rate = stats['Win Rate [%]']
    sqn = stats['SQN']  # 假设stats中包含SQN指标
    trades = stats['# Trades']  # 假设stats中包含交易次数指标

    # 对最大回撤进行惩罚（回撤越大，分数越低）
    dd_penalty = 1 / (1 + abs(max_dd/100))

    # 对交易次数进行惩罚（交易次数少于50次，分数越低）
    trade_penalty = 1 if trades >= 50 else trades / 50

    # 定义各个指标的权重
    ret_weight = 0.6         # 收益率权重
    sqn_weight = 0.4         # SQN权重
    sharpe_weight = 0.2      # 夏普比率权重
    win_rate_weight = 0.15   # 胜率权重 (新增权重)
    dd_weight = 0.1          # 回撤惩罚权重

    # 计算综合得分，包括对胜率的加权
    score = (
        ret_weight * (ret/100) +
        sqn_weight * sqn +
        sharpe_weight * sharpe +
        win_rate_weight * (win_rate/100) +
        dd_weight * dd_penalty
    ) * trade_penalty

    return score

In [5]:
def optimize_strategy(symbol, start_date, end_date, data_path, results_file, strategy_cls, optimization_params, source_timeframe, target_timeframe):
    """对单个交易对进行策略优化并及时保存结果"""
    try:
        print(f"开始加载 {symbol} 的数据...")
        # 加载数据，使用传入的时间周期参数
        df = load_and_resample_data(
            symbol=symbol,
            start_date=start_date,
            end_date=end_date,
            source_timeframe=source_timeframe,
            target_timeframe=target_timeframe
        )
        
        # 初始化回测实例，使用最新的 MeanReverterShort 策略
        bt = Backtest(
            df,
            strategy_cls,
            commission=0.0004,
            margin=1/3,
            trade_on_close=True,
            exclusive_orders=True,
            hedging=False
        )
        
        print("\n开始参数优化...")
        stats, heatmap, optimize_result = bt.optimize(
            **optimization_params,  # 直接传入 optimization_params 字典解包
            constraint=lambda p: True,
            maximize=custom_score,
            method='sambo',
            max_tries=10,
            random_state=0,
            return_heatmap=True,
            return_optimization=True
        )
        
        clear_output(wait=True)  # 清除当前 cell 的所有输出
        
        # 只提取并处理最佳的参数组合
        sorted_heatmap = heatmap.sort_values(ascending=False)
        best_params = sorted_heatmap.index[0]
        best_score = sorted_heatmap.iloc[0]
        
        current_params = dict(zip(optimization_params.keys(), best_params))
        
        final_stats = bt.run(**current_params)
        
        result = {
            'Symbol': symbol,
            'Rank': 1,
            'Initial Capital': 10000,
            'Score': best_score,
            **current_params,
            'Start': final_stats['Start'],
            'End': final_stats['End'],
            'Duration': final_stats['Duration'],
            'Exposure Time [%]': final_stats['Exposure Time [%]'],
            'Equity Final [$]': final_stats['Equity Final [$]'],
            'Equity Peak [$]': final_stats['Equity Peak [$]'],
            'Total Return (%)': final_stats['Return [%]'],
            'Buy & Hold Return (%)': final_stats['Buy & Hold Return [%]'],
            'Return (Ann.) (%)': final_stats['Return (Ann.) [%]'],
            'Volatility (Ann.) (%)': final_stats['Volatility (Ann.) [%]'],
            'CAGR (%)': final_stats.get('CAGR [%]', None),
            'Sharpe Ratio': final_stats['Sharpe Ratio'],
            'Sortino Ratio': final_stats.get('Sortino Ratio', None),
            'Calmar Ratio': final_stats.get('Calmar Ratio', None),
            'Max Drawdown (%)': final_stats['Max. Drawdown [%]'],
            'Avg Drawdown (%)': final_stats.get('Avg. Drawdown [%]', None),
            'Max Drawdown Duration': final_stats.get('Max. Drawdown Duration', None),
            'Avg Drawdown Duration': final_stats.get('Avg Drawdown Duration', None),
            'Total Trades': final_stats['# Trades'],
            'Win Rate (%)': final_stats['Win Rate [%]'],
            'Best Trade (%)': final_stats.get('Best Trade [%]', None),
            'Worst Trade (%)': final_stats.get('Worst Trade [%]', None),
            'Avg Trade (%)': final_stats.get('Avg. Trade [%]', None),
            'Max Trade Duration': final_stats.get('Max. Trade Duration', None),
            'Avg Trade Duration': final_stats.get('Avg. Trade Duration', None),
            'Profit Factor': final_stats.get('Profit Factor', None),
            'Expectancy (%)': final_stats.get('Expectancy [%]', None),
            'SQN': final_stats.get('SQN', None),
            'Kelly Criterion': final_stats.get('Kelly Criterion', None),
            'Target Timeframe': target_timeframe,  # 记录当前回测时间周期
            'Optimization Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        results = [result]
        print("完成最佳结果的处理")
        
        # 保存结果到Excel文件
        try:
            existing_df = pd.read_excel(results_file)
            new_df = pd.concat([existing_df, pd.DataFrame(results)], ignore_index=True)
        except FileNotFoundError:
            new_df = pd.DataFrame(results)
        
        new_df.to_excel(results_file, index=False)
        print(f"成功保存 {len(results)} 个优化结果")
        return results
        
    except Exception as e:
        print(f"优化 {symbol} 时出错: {str(e)}")
        print(f"错误类型: {type(e).__name__}")
        print(f"错误详情: {str(e)}")
        import traceback
        print("完整错误追踪:")
        print(traceback.format_exc())
        return None


In [6]:
def main():
    # 将所有参数集中在此处配置，便于后续维护
    config = {
        'data_path': r'\\znas\Main\futures',
        'start_date': '2024-02-05',
        'end_date': '2025-02-06',
        'source_timeframe': '1m',  # 数据源时间周期保持不变
    }
    # 设置多个目标时间周期
    target_timeframes = ['1hour', '15min', '5min', '1min']
    
    optimization_params = {
        'lowest_point_bars': range(5, 51, 5),    # 慢速 RSI 均线的周期：5,10,15,20,25,30,35,40,45,50
        'rsi_length': range(2, 51, 5),           # RSI 计算周期：2,7,12,17,22,27,32,37,42,47
        'sell_barrier': range(65, 86, 5),        # RSI 障碍水平，当启用绝对障碍时必须超过此值才平仓：65,70,75,80,85
        'dca_parts': range(4, 12, 1)             # 最大允许加仓次数（分批建仓）：4到11
    }
    
    print(f"使用数据路径: {config['data_path']}")
    print(f"回测时间范围: {config['start_date']} 到 {config['end_date']}")
    
    # 获取所有交易对
    symbols = get_all_symbols(config['data_path'], '2024-01-01')
    print(f"总共找到 {len(symbols)} 个交易对:")
    
    # 构造全局结果文件名，将策略名称与日期范围融入文件名中
    from ConnorsReversalStrategy import ConnorsReversal

    strategy_name = ConnorsReversal.__name__
    strategy_cls = ConnorsReversal
    start_clean = config['start_date'].replace("-", "")
    end_clean = config['end_date'].replace("-", "")
    master_file = f"optimization_results_{strategy_name}_{start_clean}-{end_clean}.xlsx"
    
    # 修改部分：读取已有优化结果，并生成 Symbol 与 Target Timeframe 的组合集合
    try:
        global_df = pd.read_excel(master_file)
        if not global_df.empty:
            if 'Target Timeframe' in global_df.columns:
                optimized_combinations = set(zip(global_df['Symbol'], global_df['Target Timeframe']))
                combos_str = ", ".join([f"{s}-{tf}" for s, tf in optimized_combinations])
                print(f"已有优化结果的组合: {combos_str}")
            else:
                optimized_combinations = set()
                print("警告: 'Target Timeframe' 列不存在，将略过已有结果检查，后续优化可能会重复")
        else:
            optimized_combinations = set()
            print("优化结果文件为空")
    except FileNotFoundError:
        global_df = pd.DataFrame()
        optimized_combinations = set()
        print("未找到优化结果文件，将新建文件")
    
    # 统计所有组合数量（供提示使用）
    total_combinations = [(s, tf) for s in symbols for tf in target_timeframes]
    remaining_combinations = [combo for combo in total_combinations if combo not in optimized_combinations]
    print(f"\n剩余需要优化的组合数量: {len(remaining_combinations)} 个")
    
    # 针对每个交易对进行优化（逐一遍历所有组合）
    for i, symbol in enumerate(symbols, 1):
        print(f"\n正在处理第 {i}/{len(symbols)} 个交易对: {symbol}")
        print("-" * 50)
        
        # 验证数据完整性
        print(f"验证 {symbol} 的数据完整性...")
        if not verify_data_completeness(symbol, config['start_date'], config['end_date'], config['data_path']):
            print(f"警告: 跳过 {symbol} - 数据不完整")
            continue
        print(f"{symbol} 数据完整性验证通过")
        
        # 针对每个目标时间周期进行优化
        for tf in target_timeframes:
            # 检查当前 (symbol, tf) 是否已经优化过
            if (symbol, tf) in optimized_combinations:
                print(f"组合 {symbol} 在 {tf} 时间周期下已优化，跳过。")
                continue
            
            print(f"\n开始针对时间周期 {tf} 优化 {symbol} ...")
            results = optimize_strategy(
                symbol,
                config['start_date'],
                config['end_date'],
                config['data_path'],
                master_file,
                strategy_cls,
                optimization_params,
                config['source_timeframe'],
                tf
            )
            if results:  # 每次返回一个最优结果的列表
                print("优化结果:")
                for key, value in results[0].items():
                    print(f"  {key}: {value}")
                print(f"完成 {symbol} 在 {tf} 时间周期下的优化")
            else:
                print(f"警告: {symbol} 在 {tf} 时间周期下优化失败")
    
    # 重新读取全局结果文件，统计优化结果
    try:
        final_global_df = pd.read_excel(master_file)
        if not final_global_df.empty:
            print(f"\n成功完成 {len(final_global_df)}/{len(total_combinations)} 个组合的优化")
            print(f"最终结果汇总保存于: {master_file}")
            print("\n优化结果统计:")
            print(final_global_df.describe())
        else:
            print("\n警告: 没有找到任何有效的优化结果")
    except Exception as e:
        print(f"读取最终结果文件时发生错误: {str(e)}")


In [ ]:

if __name__ == '__main__':
    main()


完成最佳结果的处理
成功保存 1 个优化结果
优化结果:
  Symbol: KASUSDT
  Rank: 1
  Initial Capital: 10000
  Score: 2.5105154558747826
  lowest_point_bars: 32
  rsi_length: 38
  sell_barrier: 85
  dca_parts: 9
  Start: 2024-02-05 00:00:00
  End: 2025-02-06 23:55:00
  Duration: 367 days 23:55:00
  Exposure Time [%]: 99.89621074879227
  Equity Final [$]: 8748.97212368
  Equity Peak [$]: 17192.210386544
  Total Return (%): -12.510278763200002
  Buy & Hold Return (%): -17.70358628888448
  Return (Ann.) (%): -12.498287473298397
  Volatility (Ann.) (%): 69.5831819578847
  CAGR (%): -12.415013789354967
  Sharpe Ratio: -0.17961649814840314
  Sortino Ratio: -0.2667539206191615
  Calmar Ratio: -0.23595109694886907
  Max Drawdown (%): -52.96982143722262
  Avg Drawdown (%): -2.195936383845601
  Max Drawdown Duration: 190 days 05:00:00
  Avg Drawdown Duration: None
  Total Trades: 4
  Win Rate (%): 25.0
  Best Trade (%): 4.2470698803864115
  Worst Trade (%): -11.098200540812753
  Avg Trade (%): -4.634331074735709
  Max Tra

: 